# Mallorn Challenge: Astronomical Time-Series Classification
## Model: Feature Engineering + LightGBM
**Author:** AI Assistant
**Dataset Structure:** `split_xx` folders containing Lightcurves (Time, Flux, Filter).

### Tổng quan
Notebook này được thiết kế riêng để xử lý dữ liệu Lightcurves (CSV):
1.  **Data Loading:** Tự động quét toàn bộ các folder `split_01`...`split_20` để gom dữ liệu.
2.  **Feature Engineering:** Trích xuất đặc trưng thống kê (Min, Max, Mean, Std, Skew) cho từng dải sóng (u, g, r, i, z, y).
3.  **Modeling:** Sử dụng **LightGBM Classifier** - thuật toán SOTA cho dữ liệu bảng/chuỗi thời gian dạng này.
4.  **Handling Imbalance:** Tự động xử lý mất cân bằng dữ liệu (Class 0 vs Class 1).

## Để Resume giữa các session khác nhau (Session A bị tắt -> Mở Session B chạy tiếp):

### Chạy Session 1: Chạy bình thường. Code trên sẽ tạo ra các file extracted_features.csv và lgb_model_fold_0.txt, lgb_model_fold_1.txt... trong phần Output.

### Lưu Session 1: Kể cả khi chưa chạy xong, nếu bạn ấn Save Version (hoặc nếu session time out nhưng có file output), Kaggle sẽ lưu lại các file đó.

### Tạo Dataset từ Output:
- Vào tab Output của Notebook Session 1.
- Ấn "New Dataset" (hoặc "Create Dataset"). Đặt tên ví dụ: mallorn-training-part1.

### Mở Session 2 (Resume):
- Mở lại notebook.
- Ở cột bên phải, ấn Add Input -> Chọn Your Datasets -> Chọn mallorn-training-part1.
- Sửa Cell 3:
CFG.RESUME_PATH = '/kaggle/input/mallorn-training-part1' 
- Ấn Run All.

Kết quả: Code sẽ tự động bỏ qua bước extract_features (load từ file CSV) và tự động bỏ qua các Fold đã có file .txt. Nó sẽ chỉ train tiếp các Fold còn thiếu.

1. Imports & Setup

In [ ]:
import os
import gc
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
import warnings

warnings.filterwarnings('ignore')

# Cấu hình hiển thị
pd.set_option('display.max_columns', 500)
plt.style.use('seaborn-v0_8-darkgrid')

print(f"LightGBM version: {lgb.__version__}")

2. CONFIG & Helper Functions

In [ ]:
class CFG:
    # Đường dẫn (Tự động tìm)
    INPUT_ROOT = '/kaggle/input'
    WORKING_DIR = '/kaggle/working'

    # Nếu chạy lần đầu: để None
    # Nếu chạy lần 2: điền đường dẫn dataset chứa output lần 1 (VD: '/kaggle/input/mallorn-part1-output')
    RESUME_PATH = None 
    # RESUME_PATH = '/kaggle/input/my-previous-output-dataset'

    # Model Params
    n_folds = 5
    seed = 42
    target_col = 'target'
    id_col = 'object_id'
    
    # LightGBM Hyperparameters
    lgb_params = {
        'objective': 'binary', # Hoặc 'multiclass' nếu > 2 lớp
        'metric': 'binary_logloss',
        'boosting_type': 'gbdt',

        # Học chậm lại để tìm điểm tối ưu tốt hơn
        'learning_rate': 0.03, 

        # Kiểm soát độ phức tạp (Tránh Overfitting)
        'num_leaves': 31,        # Model đơn giản hơn sẽ tổng quát tốt hơn
        'max_depth': -1,          # Giới hạn độ sâu cây (đang để tự do)
        'min_data_in_leaf': 20,

        # Regularization (Phạt model nếu trọng số quá lớn)
        'lambda_l1': 0.05,        # L1 Regularization
        'lambda_l2': 0.05,        # L2 Regularization

        # Sampling (Giúp model không nhìn thấy toàn bộ data mỗi lần -> chống học vẹt)
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 1,
        'n_jobs': -1,
        'verbose': -1,
        'seed': 42,
        'is_unbalance': True # Quan trọng vì dataset mất cân bằng (0 >> 1)
    }

def seed_everything(seed):
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything(CFG.seed)

# Hàm tìm dataset
def find_files():
    """Quét toàn bộ thư mục để tìm file logs và lightcurves"""
    train_log_path = None
    test_log_path = None
    sample_sub_path = None
    
    # Tìm file logs chính
    for root, dirs, files in os.walk(CFG.INPUT_ROOT):
        if 'train_log.csv' in files:
            train_log_path = os.path.join(root, 'train_log.csv')
        if 'test_log.csv' in files:
            test_log_path = os.path.join(root, 'test_log.csv')
        if 'sample_submission.csv' in files:
            sample_sub_path = os.path.join(root, 'sample_submission.csv')
            
    print(f"Train Log: {train_log_path}")
    print(f"Test Log: {test_log_path}")
    
    # Tìm các file lightcurves trong các thư mục split
    # Logic: Có thể có nhiều file trùng tên trong các split, ta sẽ gom hết
    lc_files = glob.glob(os.path.join(CFG.INPUT_ROOT, '**', '*_full_lightcurves.csv'), recursive=True)
    print(f"Found {len(lc_files)} lightcurve files.")
    
    return train_log_path, test_log_path, sample_sub_path, lc_files

train_log_path, test_log_path, sample_sub_path, lc_files = find_files()

3. Data Loading & Merging

In [ ]:
print("=== Loading Metadata (Logs) ===")
# Đọc Metadata (Labels, Z, EBV...)
if train_log_path:
    train_meta = pd.read_csv(train_log_path)
    print(f"Train Meta Shape: {train_meta.shape}")
    display(train_meta.head(3))
else:
    raise FileNotFoundError("Không tìm thấy train_log.csv!")

if test_log_path:
    test_meta = pd.read_csv(test_log_path)
    print(f"Test Meta Shape: {test_meta.shape}")
else:
    print("Warning: Test log not found.")

print("\n=== Loading Lightcurves (This may take a while) ===")
# Chiến thuật: Đọc sample để kiểm tra cột, sau đó đọc và concat
# Lưu ý: Vì file có thể rất lớn, ta chỉ đọc những cột cần thiết
dfs = []
cols_to_use = ['object_id', 'Time (MJD)', 'Flux', 'Flux_err', 'Filter']

# Thanh tiến trình đọc file
for f in tqdm(lc_files, desc="Reading LC files"):
    try:
        # Kiểm tra xem file là train hay test để tối ưu (tùy chọn)
        df_chunk = pd.read_csv(f, usecols=lambda c: c in cols_to_use)
        dfs.append(df_chunk)
    except Exception as e:
        print(f"Error reading {f}: {e}")

if dfs:
    full_lc = pd.concat(dfs, ignore_index=True)
    # Loại bỏ duplicate nếu các split chứa dữ liệu trùng lặp
    full_lc = full_lc.drop_duplicates()
    print(f"Total Lightcurve Points: {len(full_lc)}")
    display(full_lc.head())
else:
    raise ValueError("Không đọc được dữ liệu Lightcurve nào!")

4. Feature Engineering

In [ ]:
from scipy.stats import linregress

feature_file_name = 'extracted_features.csv'
loaded_features = False

# 1. Kiểm tra Resume (Session cũ)
if CFG.RESUME_PATH and os.path.exists(os.path.join(CFG.RESUME_PATH, feature_file_name)):
    print(f"🔄 Found saved features in Input: {CFG.RESUME_PATH}")
    lc_features = pd.read_csv(os.path.join(CFG.RESUME_PATH, feature_file_name))
    loaded_features = True

# 2. Kiểm tra Working Dir (Session hiện tại)
elif os.path.exists(os.path.join(CFG.WORKING_DIR, feature_file_name)):
    print(f"🔄 Found saved features in Working Directory")
    lc_features = pd.read_csv(os.path.join(CFG.WORKING_DIR, feature_file_name))
    loaded_features = True

# 3. Tính toán mới
if not loaded_features:
    print("⚡ No cached features found. Starting extraction (This takes time)...")
    
    def extract_features(lc_df):
        print("Extracting features (Phase 2: Advanced Statistics)...")
        
        # 1. Thống kê chung (Global Stats)
        aggs = {
            'Flux': [
                'min', 'max', 'mean', 'std', 'skew', 
                lambda x: np.percentile(x, 25), 
                lambda x: np.percentile(x, 50), 
                lambda x: np.percentile(x, 75), 
                lambda x: np.percentile(x, 95) - np.percentile(x, 5)
            ],
            'Flux_err': ['mean', 'max'],
            'Time (MJD)': [lambda x: x.max() - x.min(), 'count']
        }
        
        features = lc_df.groupby('object_id').agg(aggs)
        features.columns = ['_'.join(col).strip() for col in features.columns.values]
        
        rename_dict = {
            'Flux_<lambda_0>': 'flux_q25',
            'Flux_<lambda_1>': 'flux_median',
            'Flux_<lambda_2>': 'flux_q75',
            'Flux_<lambda_3>': 'flux_range90',
            'Time (MJD)_<lambda_0>': 'duration',
            'Time (MJD)_count': 'n_obs'
        }
        features.rename(columns=rename_dict, inplace=True)
        
        # 2. Pivot Table (Filter Stats) - FIX Ở ĐÂY
        # Sửa values='Flux' thành values=['Flux'] để đảm bảo có 3 levels cột
        pivot_stats = pd.pivot_table(lc_df, index='object_id', columns='Filter', 
                                     values=['Flux'], # <--- QUAN TRỌNG: Dùng list
                                     aggfunc=['mean', 'std', 'max', 'min'])
        
        # Đặt tên cột dạng: u_mean_Flux, g_std_Flux... (Filter_Agg_Value)
        # col[2]=Filter, col[1]=Agg, col[0]=Value
        pivot_stats.columns = [f"{col[2]}_{col[1]}_{col[0]}" for col in pivot_stats.columns.values]
        
        # 3. Tính Color Index
        bands = ['u', 'g', 'r', 'i', 'z', 'y']
        for i in range(len(bands)-1):
            b1, b2 = bands[i], bands[i+1]
            # Vì ta đã đặt tên cột là u_mean_Flux nên dùng startswith('u') là chuẩn
            c1 = [c for c in pivot_stats.columns if c.startswith(b1) and 'mean' in c]
            c2 = [c for c in pivot_stats.columns if c.startswith(b2) and 'mean' in c]
            
            if c1 and c2:
                features[f'color_{b1}-{b2}'] = pivot_stats[c1[0]] - pivot_stats[c2[0]]

        # Merge lại
        final_features = features.merge(pivot_stats, on='object_id', how='left')
        
        # 4. Ratios
        final_features['flux_std_over_mean'] = final_features['Flux_std'] / (final_features['Flux_mean'].abs() + 1e-6)
        final_features['amplitude'] = final_features['Flux_max'] - final_features['Flux_min']
        
        return final_features

    # Kiểm tra biến
    if 'full_lc' not in globals():
        raise NameError("Biến 'full_lc' chưa được tạo. Hãy chạy lại Cell 4!")

    # Thực thi
    lc_features = extract_features(full_lc)
    
    # Save
    save_path = os.path.join(CFG.WORKING_DIR, feature_file_name)
    lc_features.to_csv(save_path, index=True)
    print(f"💾 Features saved to: {save_path}")

    del full_lc, dfs
    gc.collect()

print(f"Features shape: {lc_features.shape}")
display(lc_features.head())

5. Prepare Train/Test Datasets

In [ ]:
print("=== Preparing Final Datasets ===")

# Merge features với Metadata (Z, EBV)
# Train Set
train_df = train_meta.merge(lc_features, on='object_id', how='left')
# Test Set
test_df = test_meta.merge(lc_features, on='object_id', how='left')

# Drop các cột không dùng train (Text, SpecType, split)
drop_cols = ['SpecType', 'English Translation', 'split', 'target', 'object_id']
# Giữ lại danh sách Feature
feature_cols = [c for c in train_df.columns if c not in drop_cols]

print(f"Features used for training ({len(feature_cols)}):")
print(feature_cols[:10], "...")

# Check target
print("Target Distribution:")
print(train_df['target'].value_counts())

# Fill NaN (LightGBM xử lý được, nhưng fill 0 hoặc mean sẽ an toàn hơn cho 1 số metrics)
# train_df = train_df.fillna(0)
# test_df = test_df.fillna(0)

6. Training Model (LightGBM)

In [ ]:
from sklearn.metrics import f1_score

print("=== Starting Training with Resume Logic ===")

X = train_df[feature_cols]
y = train_df['target']
X_test = test_df[feature_cols]

skf = StratifiedKFold(n_splits=CFG.n_folds, shuffle=True, random_state=CFG.seed)

oof_preds = np.zeros(len(X))
test_preds = np.zeros(len(X_test))
models = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"\n--- Fold {fold+1}/{CFG.n_folds} ---")
    
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
    
    # Tên file model cho Fold này
    model_filename = f"lgb_model_fold_{fold}.txt"
    
    # 1. Tìm model trong RESUME_PATH (Ưu tiên load từ session trước)
    resume_model_path = os.path.join(CFG.RESUME_PATH, model_filename) if CFG.RESUME_PATH else None
    
    model = None
    
    if resume_model_path and os.path.exists(resume_model_path):
        print(f"🔄 Resuming: Loading existing model from {resume_model_path}")
        model = lgb.Booster(model_file=resume_model_path)
    else:
        # 2. Nếu không có, Train mới
        print(f"⚡ Training New Model for Fold {fold+1}...")
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        dtrain = lgb.Dataset(X_train, label=y_train)
        dval = lgb.Dataset(X_val, label=y_val, reference=dtrain)

        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        dtrain = lgb.Dataset(X_train, label=y_train)
        dval = lgb.Dataset(X_val, label=y_val, reference=dtrain)
        
        callbacks = [
            # Tăng kiên nhẫn từ : Model sẽ đợi lâu hơn xem loss có giảm tiếp không
            lgb.early_stopping(stopping_rounds=100), 
            # In log mỗi 500 vòng cho đỡ rối mắt
            lgb.log_evaluation(period=500)
        ]

        # Train
        model = lgb.train(
            CFG.lgb_params,
            dtrain,
            valid_sets=[dtrain, dval],
            # Tăng số vòng tối đa từ để phù hợp với Learning Rate thấp
            num_boost_round=5000, 
            callbacks=callbacks
        )
        
        # Lưu model ngay sau khi train xong
        save_path = os.path.join(CFG.WORKING_DIR, model_filename)
        model.save_model(save_path)
        print(f"💾 Model saved to {save_path}")

    models.append(model)

    # Predict
    val_probs = model.predict(X_val, num_iteration=model.best_iteration)
    oof_preds[val_idx] = val_probs
    
    test_preds += model.predict(X_test, num_iteration=model.best_iteration) / CFG.n_folds
    
    # === TÌM THRESHOLD TỐI ƯU CHO FOLD NÀY ===
    best_f1 = 0
    best_thr = 0.5
    for thr in np.arange(0.1, 0.9, 0.05):
        current_f1 = f1_score(y_val, (val_probs > thr).astype(int), average='macro')
        if current_f1 > best_f1:
            best_f1 = current_f1
            best_thr = thr
            
    print(f"Fold {fold+1} Best Threshold: {best_thr:.2f} | F1-Macro: {best_f1:.4f}")

print("\n=== Training Finished ===")

# === TÌM THRESHOLD TỔNG THỂ ===
best_global_f1 = 0
best_global_thr = 0.5
for thr in np.arange(0.1, 0.9, 0.01):
    current_f1 = f1_score(y, (oof_preds > thr).astype(int), average='macro')
    if current_f1 > best_global_f1:
        best_global_f1 = current_f1
        best_global_thr = thr

print(f"\n🏆 OVERALL Best Threshold: {best_global_thr:.2f}")
print(f"🏆 OVERALL CV F1-Macro: {best_global_f1:.4f}")

# Cập nhật lại kết quả submission với ngưỡng tối ưu này
final_preds_labels = (test_preds > best_global_thr).astype(int)

    # # Generate Predictions (Dù load hay train mới đều phải predict)
    # val_pred = model.predict(X_val, num_iteration=model.best_iteration)
    # oof_preds[val_idx] = val_pred
    
    # test_pred_fold = model.predict(X_test, num_iteration=model.best_iteration)
    # test_preds += test_pred_fold / CFG.n_folds
    
    # # Quick Check
    # # Lưu ý: Ngưỡng 0.5 có thể không tối ưu cho F1 nếu data mất cân bằng
    # # Bạn có thể thử chỉnh ngưỡng này sau khi có OOF predictions
    # fold_f1 = f1_score(y_val, (val_pred > 0.5).astype(int), average='macro')
    # print(f"Fold {fold+1} F1-Macro: {fold_f1:.4f}")

print("\n=== Training Finished ===")

7. Evaluation & Feature Importance

In [ ]:
# Đánh giá tổng thể
oof_labels = (oof_preds > 0.5).astype(int)
overall_f1 = f1_score(y, oof_labels, average='macro')
overall_acc = accuracy_score(y, oof_labels)

print(f"Overall CV F1-Macro: {overall_f1:.4f}")
print(f"Overall CV Accuracy: {overall_acc:.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y, oof_labels))
print("\nClassification Report:")
print(classification_report(y, oof_labels))

# Plot Feature Importance
feature_importance = pd.DataFrame()
feature_importance["feature"] = feature_cols
feature_importance["importance"] = sum([m.feature_importance() for m in models])
feature_importance = feature_importance.sort_values(by="importance", ascending=False).head(20)

plt.figure(figsize=(10, 8))
sns.barplot(x="importance", y="feature", data=feature_importance)
plt.title("Top 20 Important Features")
plt.tight_layout()
plt.show()

8. Submission

In [ ]:
print("=== Generating Submission ===")

# Chuyển xác suất thành nhãn (0/1) nếu yêu cầu file submission là int
# Nếu yêu cầu probability, giữ nguyên test_preds
final_preds_labels = (test_preds > 0.5).astype(int)

submission = pd.DataFrame({
    'object_id': test_df['object_id'],
    'prediction': final_preds_labels 
})

# Kiểm tra format với sample_submission nếu có
if sample_sub_path:
    sample_sub = pd.read_csv(sample_sub_path)
    # Đảm bảo thứ tự object_id khớp (nếu cần thiết, thường Kaggle chấm theo ID nên merge là an toàn nhất)
    submission = submission.set_index('object_id').reindex(sample_sub['object_id']).reset_index()
    # Fill missing nếu có (phòng trường hợp test_log thiếu ID so với sample)
    submission['prediction'] = submission['prediction'].fillna(0).astype(int)

output_path = os.path.join(CFG.WORKING_DIR, 'submission.csv')
submission.to_csv(output_path, index=False)

print(f"Submission saved to: {output_path}")
print(submission.head())

In [ ]:
# import os

# files_to_delete = [
#     'extracted_features.csv', 
#     'lgb_model_fold_0.txt',
#     'lgb_model_fold_1.txt',
#     'lgb_model_fold_2.txt',
#     'lgb_model_fold_3.txt',
#     'lgb_model_fold_4.txt',
#     'submission.csv'
# ]

# print("Đang dọn dẹp file cũ...")
# for f in files_to_delete:
#     path = os.path.join('/kaggle/working', f)
#     if os.path.exists(path):
#         os.remove(path)
#         print(f"❌ Đã xóa: {f}")
#     else:
#         print(f"⚠️ Không tìm thấy: {f}")

# print("Hoàn tất. Sẵn sàng chạy code mới!")